In [13]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import imutils


ModuleNotFoundError: No module named 'imutils'

 imutils.video  will help us to work with a webcam and to calculate the estimated Frames Per Second (FPS) throughput rate   
 dlib  library will be used for its correlation tracker implementation.